In [12]:
import carball
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import torch
import torch.nn as nn
import torch.optim as optim
%matplotlib inline
cuda = False

In [15]:
from data import Calculated

data_manager = Calculated()

In [63]:
num_columns = 6
class HitPredictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(num_columns, 128),
            nn.ReLU(),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Linear(32, 10),
            nn.Sigmoid()
        )
    
    def forward(self, game_state):
        return self.layers(game_state)

In [ ]:
categories = ['pass_', 'passed', 'dribble', 'dribble_continuation', 'shot', 'goal', 'assist', 'assisted', 'save', 'aerial']
def get_train_test_data(proto, dataframe):
    # Only from the perspective of the blue team (we'll switch them up)
#     players = dataframe.columns.levels[0]
#     columns = dataframe.columns.levels[1]
    players = proto.players
    teams = proto.teams
    team_map = {}
    for i, team in enumerate(teams):
        for player in team.player_ids:
            team_map[player.id] = int(team.is_orange)
            
    name_map = {}
    for i, player in enumerate(players):
        name_map[player.id.id] = player.name
    
    hits = proto.game_stats.hits
    inputs = []
    outputs = []
    for hit in hits:
        player_name = name_map[hit.player_id.id]
        
        frame = dataframe.iloc[hit.frame_number]
        
        player_location = frame[player_name]
        ball_location = frame['ball']
        input_vector = np.append(ball_location[['pos_x', 'pos_y', 'pos_z']], player_location[['pos_x', 'pos_y', 'pos_z']])
        
        inputs.append(input_vector)
        vector = np.zeros(len(categories))
        for i, c in enumerate(categories):
            if getattr(hit, c):
                vector[i] = 1
        outputs.append(vector)
    inputs = np.array(inputs).astype(float)
    outputs = np.array(outputs)
    rand = np.random.rand(outputs.shape[0])
    threshold = rand < 0.8
    input_train, input_test = inputs[threshold], inputs[~threshold]
    output_train, output_test = outputs[threshold], outputs[~threshold]
    
    return input_train, input_test, output_train, output_test


loss = nn.BCELoss()
# loss = nn.MSELoss()
def train_on_proto_df(proto, df, model):
    input_train, input_test, output_train, output_test = get_train_test_data(proto, df)
    if input_train is None:
        print('data is none')
        return
    input_tensor = torch.from_numpy(input_train).float()
    output_tensor = torch.from_numpy(output_train).float()
    if cuda:
        input_tensor = input_tensor.cuda()
        output_tensor = output_tensor.cuda()
#     print(output_train.shape, output_test.shape)
    print(proto.game_metadata.match_guid)
    for x in range(100):
        opt.zero_grad()
        predicted = model(input_tensor)
#         print("shapes", predicted.shape, output_tensor.shape)
        loss_val = loss(predicted, output_tensor)
        loss_val.backward()
        opt.step()
        if x % 30 == 0:
#             print(input_tensor.shape, predicted.shape, output_tensor.shape)
#             print('Loss:', loss_val)
            input_test_tensor = torch.from_numpy(input_test).float()
            if cuda:
                input_test_tensor = input_test_tensor.cuda()
            test_output = model(input_test_tensor)
            output = test_output.cpu().detach().numpy()
            accuracy = (output.round() == output_test)
    return loss_val.cpu().data.numpy(), accuracy

In [ ]:
model = HitPredictor().train()
if cuda:
    model = model.cuda()
opt = optim.Adam(model.parameters(), lr=1e-4)
epochs = 25
epoch_losses = []
accuracy = []
for epoch in range(epochs):
    epoch_loss = []
    epoch_accuracy = []
    for n in range(1):
        for replay in data_manager.get_replay_list(num=100, page=n + 1):
            df = data_manager.get_pandas(replay)
            proto = data_manager.get_proto(replay)
            if df is None or proto is None:
                continue
            try:
                los, acc = train_on_proto_df(proto, df, model)
            except Exception as e:
                print(e)
                continue
            epoch_loss.append(los)
            epoch_accuracy.append(acc)
    epoch_losses.append(epoch_loss)
    accuracy.append(epoch_accuracy)

In [ ]:
means = []
for l in epoch_losses:
    l = np.array(l)
    l = l[l != None]
    means.append(l.mean())

plt.plot(range(len(epoch_losses)), means)
plt.title('Loss')

In [ ]:
# [epoch#, replay#, hit#, category#]
means = []
for epoch in accuracy:
    epoch = np.asarray(np.asarray(epoch).tolist())
    replay_means = []
    for replay in epoch:
        cat_means = []
        for i, c in enumerate(categories):
            cat_means.append(np.array(replay[:, i].mean()))
        replay_means.append(np.array(cat_means))
    means.append(np.array(replay_means))
means = np.array(means)

for i, c in enumerate(categories):
    data = [means[epoch][:, i].mean() for epoch in range(len(accuracy))]
    plt.plot(data)
plt.legend(categories)
# plt.plot(range(len(accuracy)), means)
# plt.title('Accuracy')
#plt.legend(['Team Accuracy', 'Frame Accuracy']);